In [1]:
import os
import json
# !pip install spotipy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt

In [2]:
#The Goal: 
# A supervised learning algorithm, such as a decision tree or support vector machine, 
# could be used to classify the genre of music based on the data provided. 
# The algorithm would be trained on a dataset of labeled music genres and the associated data values, 
# and then it could be used to predict the genre of music given a new set of data values.

cid ='d8762235995b4ddcbc0df90739df9a63' # Client ID; copy this from your app created on beta.developer.spotify.com
secret = '887a4eccdbde4195bfc0f7f4c472fe70' # Client Secret; copy this from your app
username = '12155311349' # Your Spotify username

#for avaliable scopes see https://developer.spotify.com/web-api/using-scopes/
scope = 'user-library-read playlist-modify-public playlist-read-private'

redirect_uri='https://localhost:8080/callback/'

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret) 

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

token = util.prompt_for_user_token(username,scope,client_id=cid,client_secret=secret,redirect_uri="https://localhost:8080/callback/")

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)


In [3]:
#Data has already been created and saved. Check how it is done in AI_Random_Forest to see where the data comes from. It's necessary to save time. 
tracks = pd.read_csv('tracks.csv')
artists = pd.read_csv('artists.csv')
# full_data_random = pd.read_csv('full_data_random.csv')
# num_data_random = pd.read_csv('num_data_random.csv')

In [4]:
artists

,id,followers,genres,name,popularity
0,0DheY5irMjBUeLybbCUEZ2,0.0,[],Armid & Amir Zare Pashai feat. Sara Rouzbehani,0
1,0DlhY15l3wsrnlfGio2bjU,5.0,[],ปูนา ภาวิณี,0
2,0DmRESX2JknGPQyO15yxg7,0.0,[],Sadaa,0
3,0DmhnbHjm1qw6NCYPeZNgJ,0.0,[],Tra'gruda,0
4,0Dn11fWM7vHQ3rinvWEl4E,2.0,[],Ioannis Panoutsopoulos,0
...,...,...,...,...,...
1162090,3cOzi726Iav1toV2LRVEjp,4831.0,['black comedy'],Ali Siddiq,34
1162091,6LogY6VMM3jgAE6fPzXeMl,46.0,[],Rodney Laney,2
1162092,19boQkDEIay9GaVAWkUhTa,257.0,[],Blake Wexler,10
1162093,5nvjpU3Y7L6Hpe54QuvDjy,2357.0,['black comedy'],Donnell Rawlings,15


In [5]:
tracks.head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,['Uli'],['45tIt06XoI0Iio4LBEVpls'],1922-02-22,0.645,0.4450,0,-13.338,1,0.4510,0.674,0.7440,0.151,0.127,104.851,3
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,['Fernando Pessoa'],['14jtPCOoNZwquk5wd9DxrY'],1922-06-01,0.695,0.2630,0,-22.136,1,0.9570,0.797,0.0000,0.148,0.655,102.009,1
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.434,0.1770,1,-21.180,1,0.0512,0.994,0.0218,0.212,0.457,130.418,5
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.9180,0.104,0.397,169.980,3
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,['Dick Haymes'],['3BiJGZsyX9sJchTqcSA7Su'],1922,0.402,0.1580,3,-16.900,0,0.0390,0.989,0.1300,0.311,0.196,103.220,4


In [6]:
#Sorting through the junk;reseting the index to allow it to sort correct
tracks_filtered = tracks[tracks['popularity'] >= 70].reset_index(drop=True)
# tracks_filtered

#Lowering Dataset to get rid of artists with less than 100000 followers
artists_filtered = artists[artists['followers'] >= 100000].reset_index(drop=True)
artists_filtered
# tracks_filtered

,id,followers,genres,name,popularity
0,72578usTM6Cj5qWsi471Nc,248568.0,"['filmi', 'indian folk', 'indian rock', 'kanna...",Raghu Dixit,52
1,7b6Ui7JVaBDEfZB9k6nHL0,701766.0,"['desi pop', 'hindi indie', 'indian indie', 'i...",The Local Train,57
2,5wJ1H6ud777odtZl5gG507,211174.0,"['desi pop', 'modern bollywood']",Vishal Mishra,66
3,0n4a5imdLBN24fIrBWoqrv,328492.0,"['opm', 'pinoy hip hop', 'pinoy r&b', 'pinoy t...",Because,59
4,4gdMJYnopf2nEUcanAwstx,103905.0,"['hindi indie', 'indian indie', 'indian singer...",Anuv Jain,58
...,...,...,...,...,...
13264,7dFJWWaHc5Z8oQbIRgBcwQ,115362.0,"['alabama rap', 'deep southern trap', 'trap']",Doe B,42
13265,28ie4NNTa2VW2QV4Zray8M,269262.0,"['funk carioca', 'funk ostentacao']",Mc Poze do Rodo,67
13266,2KUMXzsDcJqXZW0CRh07zP,102559.0,['v-pop'],Uyên Linh,50
13267,4ZWvN9FEfdTea1SEHjpTNi,109770.0,"['indie pop', 'indie rock']",Pinback,51


In [7]:
tracks_filtered

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,78MI7mu1LV1k4IA2HzKmHe,Dream A Little Dream Of Me - Single Version,74,185160,0,"['Ella Fitzgerald', 'Louis Armstrong']","['5V0MlUE1Bft0mbLlND7FJz', '19eLuQmk9aCobbVDHc...",1957-01-01,0.443,0.104,0,-17.042,1,0.1010,0.913,0.000000,0.1910,0.3940,76.497,4
1,6VTbbVjKOC2qWagIDbkJrC,My Baby Just Cares for Me - 2013 Remastered Ve...,71,215747,0,['Nina Simone'],['7G1GBhoKtEPnP86X2PvEYO'],1958-06-24,0.746,0.243,4,-15.134,0,0.1230,0.829,0.000001,0.0905,0.4250,118.578,4
2,4gphxUgq0JSFv2BCLhNDiE,Jailhouse Rock,71,146480,0,['Elvis Presley'],['43ZHCT0cAZBISjO8DG9PnE'],1958-03-21,0.647,0.582,10,-9.538,0,0.0755,0.410,0.000002,0.0715,0.9150,167.396,4
3,2QfiRTz5Yc8DdShCxG1tB2,Johnny B. Goode,77,161560,0,['Chuck Berry'],['293zczrfYafIItmnmM3coR'],1959-07-01,0.534,0.803,10,-9.129,1,0.0743,0.741,0.000061,0.3070,0.9690,167.983,4
4,0AQquaENerGps8BQmbPw14,Big Iron,70,235733,0,['Marty Robbins'],['0Xi59sEw38vRvwleSAVqoo'],1959,0.726,0.474,1,-7.834,0,0.0491,0.803,0.000000,0.2580,0.5700,116.027,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7318,3R9KvtovDyPBPtBqiulmoq,1942,70,185263,1,['Ant Wan'],['51TXQniEQkYh89tfLjiuSy'],2021-02-25,0.720,0.470,10,-6.922,0,0.2240,0.626,0.000037,0.1380,0.2080,85.496,4
7319,44r4zta6P9flkhKaVnbsvG,Freaks,70,174800,0,['Jordan Clarke'],['14Y3trk7LaslSFTk1G35rx'],2019-02-25,0.674,0.568,1,-6.356,1,0.0408,0.116,0.000000,0.1650,0.6400,166.107,4
7320,1uviKYHZuM4uINK33F7sCt,Fix It to Break It,70,198799,0,['Clinton Kane'],['7okSU80WTrn4LXlyXYbX3P'],2020-03-27,0.493,0.461,2,-8.524,1,0.0456,0.845,0.000000,0.1150,0.3500,51.414,4
7321,0NuWgxEp51CutD2pJoF4OM,blind,72,153293,0,['ROLE MODEL'],['1dy5WNgIKQU6ezkpZs4y8z'],2020-10-21,0.765,0.663,0,-5.223,1,0.0652,0.141,0.000297,0.0924,0.6860,150.091,4


In [8]:
#Making a copy of our tracks_filtered to save the genres too each song
tracks_copy = tracks_filtered.copy()

#Filtering through to add the genre to each song; first we search in artists for the related id of a song
for i in range(len(tracks_filtered)):
    artist_id = str(tracks_filtered['id_artists'][i]).replace("[", "").replace("]", "").replace("'", "")
    filtered_df = artists_filtered[artists_filtered['id'] == artist_id]
    if len(filtered_df) > 0:
        tracks_copy.loc[i, 'genre'] = filtered_df.iloc[0]['genres']

In [9]:
# #Need to lower the amount of genres
# for i in range(len(artists_filtered)):
#     first_genre = artists_filtered['genres'][i].split(',')[0].strip("['").strip("']")
#     if len(filtered_df) > 0:
#         tracks_copy.loc[i, 'genre'] = filtered_df.iloc[0]['genres']

# tracks_copy

In [10]:
#Filtering the data to get rid of NaN Values
tracks_copy = tracks_copy[tracks_copy['genre'] >= 'NaN'].reset_index(drop=True)
# tracks_copy = tracks_copy[tracks_copy['genre'] == ' '].reset_index(drop=True)

In [11]:
#Search for a certain genre within and then we need to assign that genre to each song to lower the amount and make it more predictable
def genre_sort(genre):
    tracks_copy[genre] = tracks_copy['genre'].apply(lambda x: genre in x)
    #if pop = true add the pop tag
    tracks_copy['genre'] = genre #Country
    t=tracks_copy.drop(genre, axis=1)
    return t

In [12]:
# pop = genre_sort('pop')
# rap = genre_sort('rap')
# country = genre_sort('country')
# rock = genre_sort('rock')
# jazz = genre_sort('jazz')
# metal = genre_sort('metal')
# indie_rock = genre_sort('indie rock')
# blues = genre_sort('blues')
# classical = genre_sort('classical')
# trap = genre_sort('trap')
# punk= genre_sort('punk')
# edm = genre_sort('edm')
# pop_rock= genre_sort('pop rock')
# reggae = genre_sort('reggaeton')
# electronic = genre_sort('electronic')
# folk = genre_sort('folk')
# dance_pop = genre_sort('dance pop')
# classic_rock = genre_sort('classic rock')
# pop_rap = genre_sort('pop rap')
dance = genre_sort('dance')

           

In [13]:
dance

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,genre
0,6VTbbVjKOC2qWagIDbkJrC,My Baby Just Cares for Me - 2013 Remastered Ve...,71,215747,0,['Nina Simone'],['7G1GBhoKtEPnP86X2PvEYO'],1958-06-24,0.746,0.243,...,-15.134,0,0.1230,0.8290,0.000001,0.0905,0.4250,118.578,4,dance
1,4gphxUgq0JSFv2BCLhNDiE,Jailhouse Rock,71,146480,0,['Elvis Presley'],['43ZHCT0cAZBISjO8DG9PnE'],1958-03-21,0.647,0.582,...,-9.538,0,0.0755,0.4100,0.000002,0.0715,0.9150,167.396,4,dance
2,2QfiRTz5Yc8DdShCxG1tB2,Johnny B. Goode,77,161560,0,['Chuck Berry'],['293zczrfYafIItmnmM3coR'],1959-07-01,0.534,0.803,...,-9.129,1,0.0743,0.7410,0.000061,0.3070,0.9690,167.983,4,dance
3,0AQquaENerGps8BQmbPw14,Big Iron,70,235733,0,['Marty Robbins'],['0Xi59sEw38vRvwleSAVqoo'],1959,0.726,0.474,...,-7.834,0,0.0491,0.8030,0.000000,0.2580,0.5700,116.027,4,dance
4,4Hhv2vrOTy89HFRcjU3QOx,At Last,77,179693,0,['Etta James'],['0iOVhN3tnSvgDbcg25JoJb'],1960,0.274,0.348,...,-8.631,1,0.0293,0.5470,0.013300,0.3340,0.3280,87.430,3,dance
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4741,7t3VznmzfrDXFyexb2wA0q,Every Minute,71,182507,0,['Eric Saade'],['4zGH6gaO4NohcfCXZ31bQG'],2021-02-27,0.562,0.830,...,-5.358,0,0.1630,0.0313,0.000000,0.2110,0.5730,119.879,4,dance
4742,0r6uP3bakGB2SlfA5JEotS,Mimosa,70,171845,1,['Dree Low'],['3tNPDodRNuLdezJnTsYOqy'],2021-03-25,0.800,0.521,...,-9.362,0,0.2240,0.3330,0.000002,0.0981,0.3960,102.991,4,dance
4743,1uviKYHZuM4uINK33F7sCt,Fix It to Break It,70,198799,0,['Clinton Kane'],['7okSU80WTrn4LXlyXYbX3P'],2020-03-27,0.493,0.461,...,-8.524,1,0.0456,0.8450,0.000000,0.1150,0.3500,51.414,4,dance
4744,0NuWgxEp51CutD2pJoF4OM,blind,72,153293,0,['ROLE MODEL'],['1dy5WNgIKQU6ezkpZs4y8z'],2020-10-21,0.765,0.663,...,-5.223,1,0.0652,0.1410,0.000297,0.0924,0.6860,150.091,4,dance


In [14]:
# #Need to lower the amount of genres
# less_genres = tracks_copy.copy()
# for i in range(len(tracks_copy)):
#     first_genre = tracks_copy['genre'][i].split(',')[0].strip("['").strip("']")
#     if len(tracks_copy) > 0:
#         less_genres.loc[i, 'genre'] = first_genre
# # less_genres 

In [15]:
#If the genre has less than 25 songs within it then we get rid of it
threshold = 35   # set the threshold count
value_counts = less_genres['genre'].value_counts()   # calculate the value counts
less_genres_2 = less_genres.loc[less_genres['genre'].isin(value_counts.index[value_counts > threshold])]   # filter the DataFrame based on the threshold
less_genres_2['genre'].value_counts()

NameError: name 'less_genres' is not defined

In [ ]:
num_user_playlist = less_genres_2[['danceability','energy','key','loudness','mode','speechiness','acousticness','instrumentalness','liveness','valence','tempo','time_signature','genre']]
# num_user_playlist['genre']

In [ ]:
num_user_playlist =num_user_playlist.sample(frac=1)
num_user_playlist

In [ ]:
#Modeling
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint
from sklearn.preprocessing import StandardScaler
from sklearn import tree



In [ ]:
# Split the data into features (X) and target (y)
X = num_user_playlist.drop('genre', axis=1)
y = num_user_playlist['genre']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Scale the features using StandardScaler
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

In [ ]:
#Fitting and Evaluating the Model 
rf = RandomForestClassifier(n_estimators = 400, max_depth = 150)
rf.fit(X_train, y_train)

In [ ]:
# plot the first decision tree in the random forest
fig, axes = plt.subplots(nrows = 1,ncols = 1,figsize = (4,4), dpi=800)
tree.plot_tree(rf.estimators_[0],
               feature_names = X_train.columns,
               class_names=y_train.unique(),
               filled = True,
              max_depth =1);

# show the plot
plt.show()

In [ ]:
y_pred = rf.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
import pickle
with open('Spotify_dataset.pkl', 'wb') as f:
    pickle.dump(rf, f)

In [ ]:
param_dist = {'n_estimators': randint(25,500),
              'max_depth': randint(1,230)}

# Create a random forest classifier
rf = RandomForestClassifier()

# Use random search to find the best hyperparameters
rand_search = RandomizedSearchCV(rf, 
                                 param_distributions = param_dist, 
                                 n_iter=5, 
                                 cv=5)

# Fit the random search object to the data
rand_search.fit(X_train, y_train)

In [ ]:
# Create a variable for the best model
best_rf = rand_search.best_estimator_

# Print the best hyperparameters
print('Best hyperparameters:',  rand_search.best_params_)

In [ ]:
# Generate predictions with the best model
y_pred = best_rf.predict(X_test)

# Create the confusion matrix
cm = confusion_matrix(y_test, y_pred)

ConfusionMatrixDisplay(confusion_matrix=cm).plot();

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
# precision = precision_score(y_test, y_pred)
# recall = recall_score(y_test, y_pred)

print("Accuracy:", accuracy)
# print("Precision:", precision)
# print("Recall:", recall)

In [ ]:
# Create a series containing feature importances from the model and feature names from the training data
feature_importances = pd.Series(best_rf.feature_importances_, index=X_train.columns).sort_values(ascending=False)

# Plot a simple bar chart
feature_importances.plot.bar();

In [ ]:
y_pred

In [ ]:
#Printing out our y_test
predicted = []
for i in range(0, len(y_test)):
    f=tracks_copy.loc[y_test.index.values[i],:] 
    predicted.append(f)

predicted = pd.DataFrame(predicted) 
predicted[['name','genre',]]

In [ ]:
# predicted.to_csv('AI_Results.csv', index=False)
print('Total Songs: ', predicted.index.size)
print('Country: ',  predicted[predicted['genre'] == 'country'].index.size)
print('Rock: ',  predicted[predicted['genre'] == 'rock'].index.size)
print('Rap: ',  predicted[predicted['genre'] == 'rap'].index.size)
print('RnB: ',  predicted[predicted['genre'] == 'rnb'].index.size)
print('Pop: ',  predicted[predicted['genre'] == 'pop'].index.size)
print('Classical: ',  predicted[predicted['genre'] == 'classical'].index.size)


In [ ]:
#List the Country Songs Predicted
country_pred = predicted[predicted['genre'] == 'country']
country_pred[['Artist','Song Title','genre',]]

In [ ]:
# BOXPLOTS of other data
def boxPlot(playlist, genre):
    playlist_num = playlist.drop(columns = ['mode','type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms',
       'time_signature', 'genre', 'Song Title', 'Artist'])
    fig, ax = plt.subplots(figsize=(14,7))
    sns.boxplot(data = playlist_num)
    plt.xlabel(genre, fontweight="bold")
    plt.ylabel('Value', fontweight="bold")
    return plt


In [ ]:
country_boxplot = boxPlot(country,'Country')
rock_boxplot = boxPlot(rock,'Rock')
rap_boxplot = boxPlot(rap,'Rap')
rnb_boxplot = boxPlot(rnb,'R&B')
pop_boxplot = boxPlot(pop,'POP')
classical_boxplot = boxPlot(classical,'Classical')

In [ ]:
# Plot linear correlation matrix
def linear_correlation(playlist,genre):
    playlist_num = playlist.drop(columns = ['mode','type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms',
       'time_signature', 'genre', 'Song Title', 'Artist'])
    fig, ax = plt.subplots(figsize=(14,7))
    sns.heatmap(playlist_num.corr(), annot=True, cmap='YlGnBu', vmin=-1, vmax=1, center=0, ax=ax)
    plt.title(genre +' Linear Correlation Matrix')
    plt.show()

In [ ]:
linear_correlation(country,'Country')
linear_correlation(rock,'Rock')
linear_correlation(rap,'Rap')
linear_correlation(rnb,'R&B')
linear_correlation(pop,'POP')
linear_correlation(classical,'Classical')

In [ ]:
import plotly.graph_objects as go
import plotly.offline as pyo

def radar_plot(playlist,genre,color):
    playlist_num = playlist.drop(columns = ['mode','type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms',
           'time_signature', 'genre', 'Song Title', 'Artist'])

    categories = playlist_num.columns
    categories = [*categories, categories[0]]

    mean_r = playlist_num.mean()

    mean_r = [*mean_r, mean_r[0]]


    fig = go.Figure(
        data=[
            go.Scatterpolar(r=mean_r, theta=categories, name= genre,fill='toself',fillcolor = color, ),],
        layout=go.Layout(
            title=go.layout.Title(text= genre+' Mean Values Radar Plot'),
            polar={'radialaxis': {'visible': True}},
            showlegend=True,
        )
    )

    pyo.iplot(fig)

In [ ]:
radar_plot(country,'Country','orange')
radar_plot(rock,'Rock','red')
radar_plot(rap,'Rap','lightblue')
radar_plot(rnb,'R&B','purple')
radar_plot(pop,'POP','pink')
radar_plot(classical,'Classical','green')